### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [39]:
# Dependencies and Setup
import pandas as pd
import os 
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head(3)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [40]:
#Total number of schools
schools = school_data_complete["school_name"].unique()
total_schools = len(schools)

#Total number of students
students = school_data_complete["Student ID"].unique()
total_students = len(students)

#Total budget
total_budget = "${:,.2f}".format(school_data["budget"].sum())

#Average math score
avg_math = school_data_complete["math_score"].mean()

#Average reading score
avg_reading = school_data_complete["reading_score"].mean()

#Percentage of students with a passing math score (70 or greater)
math_passing = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["Student ID"]
math_passing_percent = math_passing / total_students

#Percentage of students with a passing reading score (70 or greater)
reading_passing = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["Student ID"]
reading_passing_percent = reading_passing / total_students

#Percentage of students who passed math and reading (% Overall Passing)
both_passing = school_data_complete[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)].count()["Student ID"]
both_passing_percent = both_passing / total_students 

#Dataframe to hold the above results with cleaner format
district_summary = {"Total Number of Schools" : [total_schools], 
                    "Total Number of Students" : ["{:,}".format(total_students)],
                    "Total Budget" : [total_budget], 
                    "Average Math Score" : [round(avg_math, 2)], 
                    "Average Reading Score" : [round(avg_reading,2)], 
                    "% Passing Math" : [round(math_passing_percent * 100,2)],
                    "% Passing Reading" : [round(reading_passing_percent * 100, 2)],
                    "% Passing Math and Reading" : [round(both_passing_percent * 100,2)] }

district_summary_df = pd.DataFrame (district_summary)
district_summary_df


,Total Number of Schools,Total Number of Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Math and Reading
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [41]:

#Budget per student
per_student_budget = school_data["budget"]/school_data["size"]
school_data["Per Student Budget"] = per_student_budget

#Average math and reading score
average_scores = round(student_data.groupby(["school_name"]).mean(),2)
del average_scores["Student ID"]
#average_scores

school_summary = pd.merge(school_data, average_scores, on="school_name", how="left")
school_summary = school_summary.rename(columns = {"reading_score" : "Avg Reading Score", "math_score" : "Avg Math Score"})

#% Passing math 
passing_math = school_data_complete.loc[school_data_complete["math_score"] >= 70]
passing_math2 = passing_math.groupby(["school_name"]).count()
passing_math_total = passing_math2.filter(["school_name", "math_score"])

school_summary = pd.merge(school_summary, passing_math_total, on="school_name", how="left")
school_summary["% Passing Math"] = round((school_summary["math_score"] / school_summary["size"])*100,2)

#% Passing reading 
passing_reading = school_data_complete.loc[school_data_complete["reading_score"] >= 70]
passing_reading2 = passing_reading.groupby(["school_name"]).count()
passing_reading_total = passing_reading2.filter(["school_name", "reading_score"])

school_summary = pd.merge(school_summary, passing_reading_total, on="school_name", how="left")
school_summary["% Passing Reading"] = round((school_summary["reading_score"] / school_summary["size"])*100,2)
del school_summary["math_score"]
del school_summary["reading_score"]


#% Overall passing 
passing_overall = passing_math.loc[passing_math["reading_score"] >= 70]
passing_overall2 = passing_overall.groupby(["school_name"]).count()
passing_overall_total = passing_overall2.filter(["school_name", "math_score"])

school_summary = pd.merge(school_summary, passing_overall_total, on="school_name", how="left")
school_summary["% Passing Overall"] = round((school_summary["math_score"] / school_summary["size"])*100,2)
del school_summary ["math_score"]


school_summary = school_summary.rename (columns = {"school_name" : "School Name", 
                                                  "type" : "Type", 
                                                   "size" : "Size",
                                                  "budget" : "Budget"})
school_summary["Budget"] = school_summary["Budget"].map('${:,.2f}'.format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map('${:,.2f}'.format)

del school_summary["School ID"]
school_summary

,School Name,Type,Size,Budget,Per Student Budget,Avg Reading Score,Avg Math Score,% Passing Math,% Passing Reading,% Passing Overall
0,Huang High School,District,2917,"$1,910,635.00",$655.00,81.18,76.63,65.68,81.32,53.51
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,81.16,76.71,65.99,80.74,53.20
2,Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.73,83.36,93.87,95.85,89.89
3,Hernandez High School,District,4635,"$3,022,020.00",$652.00,80.93,77.29,66.75,80.86,53.53
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.82,83.35,93.39,97.14,90.60
5,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.99,83.27,93.87,96.54,90.58
6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.98,83.06,94.13,97.04,91.33
7,Bailey High School,District,4976,"$3,124,928.00",$628.00,81.03,77.05,66.68,81.93,54.64
8,Holden High School,Charter,427,"$248,087.00",$581.00,83.81,83.80,92.51,96.25,89.23
9,Pena High School,Charter,962,"$585,858.00",$609.00,84.04,83.84,94.59,95.95,90.54


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [42]:
school_summary_ranking = school_summary.sort_values (["% Passing Overall"], ascending=False)
school_summary_ranking.head(5)

,School Name,Type,Size,Budget,Per Student Budget,Avg Reading Score,Avg Math Score,% Passing Math,% Passing Reading,% Passing Overall
6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.98,83.06,94.13,97.04,91.33
14,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.85,83.42,93.27,97.31,90.95
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.82,83.35,93.39,97.14,90.60
5,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.99,83.27,93.87,96.54,90.58
9,Pena High School,Charter,962,"$585,858.00",$609.00,84.04,83.84,94.59,95.95,90.54


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [43]:
school_summary_ranking = school_summary.sort_values (["% Passing Overall"], ascending=True)
school_summary_ranking.head(5)

,School Name,Type,Size,Budget,Per Student Budget,Avg Reading Score,Avg Math Score,% Passing Math,% Passing Reading,% Passing Overall
11,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,80.74,76.84,66.37,80.22,52.99
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,81.16,76.71,65.99,80.74,53.20
0,Huang High School,District,2917,"$1,910,635.00",$655.00,81.18,76.63,65.68,81.32,53.51
3,Hernandez High School,District,4635,"$3,022,020.00",$652.00,80.93,77.29,66.75,80.86,53.53
12,Johnson High School,District,4761,"$3,094,650.00",$650.00,80.97,77.07,66.06,81.22,53.54


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [44]:
math_9th = school_data_complete.loc[school_data_complete["grade"] == "9th"]
math_9th = round(math_9th.groupby("school_name").mean(),2)

math_9th = math_9th.filter(["school_name", "math_score"])
math_9th = math_9th.rename(columns = {"math_score" : "9th"})

math_10th = school_data_complete.loc[school_data_complete["grade"] == "10th"]
math_10th = round(math_10th.groupby("school_name").mean(),2)

math_10th = math_10th.filter(["school_name", "math_score"])
math_10th = math_10th.rename(columns = {"math_score" : "10th"})

math_scores = pd.merge(math_9th, math_10th, on="school_name", how="inner")

math_11th = school_data_complete.loc[school_data_complete["grade"] == "11th"]
math_11th = round(math_11th.groupby("school_name").mean(),2)

math_11th = math_11th.filter(["school_name", "math_score"])
math_11th = math_11th.rename(columns = {"math_score" : "11th"})

math_scores = pd.merge(math_scores, math_11th, on="school_name", how="inner")

math_12th = school_data_complete.loc[school_data_complete["grade"] == "12th"]
math_12th = round(math_12th.groupby("school_name").mean(),2)

math_12th = math_12th.filter(["school_name", "math_score"])
math_12th = math_12th.rename(columns = {"math_score" : "12th"})

math_scores = pd.merge(math_scores, math_12th, on="school_name", how="inner")
math_scores


,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [45]:
read_9th = school_data_complete.loc[school_data_complete["grade"] == "9th"]
read_9th = round(read_9th.groupby("school_name").mean(),2)

read_9th = read_9th.filter(["school_name", "reading_score"])
read_9th = read_9th.rename(columns = {"reading_score" : "9th"})

read_10th = school_data_complete.loc[school_data_complete["grade"] == "10th"]
read_10th = round(read_10th.groupby("school_name").mean(),2)

read_10th = read_10th.filter(["school_name", "reading_score"])
read_10th = read_10th.rename(columns = {"reading_score" : "10th"})

reading_scores = pd.merge(read_9th, read_10th, on="school_name", how="inner")

read_11th = school_data_complete.loc[school_data_complete["grade"] == "11th"]
read_11th = round(read_11th.groupby("school_name").mean(),2)

read_11th = read_11th.filter(["school_name", "reading_score"])
read_11th = read_11th.rename(columns = {"reading_score" : "11th"})

reading_scores = pd.merge(reading_scores, read_11th, on="school_name", how="inner")

read_12th = school_data_complete.loc[school_data_complete["grade"] == "12th"]
read_12th = round(read_12th.groupby("school_name").mean(),2)

read_12th = read_12th.filter(["school_name", "reading_score"])
read_12th = read_12th.rename(columns = {"reading_score" : "12th"})

reading_scores = pd.merge(reading_scores, read_12th, on="school_name", how="inner")
reading_scores


,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [72]:
bins = [0, 584, 629, 644, 675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]

working_file = pd.merge(student_data, school_data, on="school_name", how="left")
working_file = working_file.filter(["student_name", "reading_score", "math_score", "size", "budget"])
working_file["Per Student Budget"] = working_file["budget"] / working_file["size"]
working_file["Passed Math"] = np.where(working_file["math_score"] >= 70, 1, 0)
working_file["Passed Reading"] = np.where(working_file["reading_score"] >= 70, 1, 0)
working_file["Passed Overall"] = np.where((working_file["math_score"] >= 70) & (working_file["reading_score"] >= 70), 1, 0)

working_file ["Budget Group"] = pd.cut(working_file["Per Student Budget"], bins=bins, labels=group_names)

grouped_file = working_file.groupby("Budget Group")

total_students_by_size = grouped_file["student_name"].count()
average_scores = grouped_file[["math_score", "reading_score"]].mean()
passing_totals = grouped_file[["Passed Math", "Passed Reading", "Passed Overall"]].sum()

budget_summary = pd.merge(average_scores, passing_totals, on="Budget Group", how="outer")
budget_summary = pd.merge(budget_summary, total_students_by_size, on="Budget Group", how="outer")
budget_summary["% Passing Math"] = round(budget_summary["Passed Math"] / budget_summary["student_name"] * 100,2)
budget_summary["% Passing Reading"] = round(budget_summary["Passed Reading"] / budget_summary["student_name"] * 100,2)
budget_summary["% Passing Overall"] = round(budget_summary["Passed Overall"] / budget_summary["student_name"] * 100,2)

budget_summary = budget_summary.rename( columns={ "math_score" : "Average Math Score", 
                                            "reading_score" : "Average Reading Score"})
budget_summary = budget_summary.filter( ["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Passing Overall"])

budget_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Budget Group,,,,,
<$584,83.363065,83.964039,93.70,96.69,90.64
$585-629,79.982873,82.312643,79.11,88.51,70.94
$630-644,77.821056,81.301007,70.62,82.60,58.84
$645-675,77.049297,81.005604,66.23,81.11,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [73]:
bins = [0, 999, 1999, 4999]
group_names = ["Small (<1000)", "Medium (1000 - 2000)", "Large (2000 - 5000)"]

workingfile = school_data_complete.filter(["math_score", "reading_score", "size"])
workingfile["passing_math"] = np.where(working_file["math_score"] >= 70, 1, 0)
workingfile["passing_reading"]= np.where(working_file["reading_score"] >= 70, 1, 0)
workingfile["passing_overall"] = np.where((working_file["math_score"] >= 70) & (working_file["reading_score"] >= 70), 1, 0)


workingfile["Size Group"] = pd.cut(workingfile["size"], bins = bins, labels = group_names)

groupedfile = workingfile.groupby("Size Group")

average_scores = groupedfile[["math_score", "reading_score"]].mean()
total_students = groupedfile["math_score"].count()
passing_totals = groupedfile[["passing_math", "passing_reading", "passing_overall"]].sum()

size_summary = pd.merge(average_scores, total_students, on="Size Group", how="outer")
size_summary = size_summary.rename (columns={"math_score_y" : "Total Students", 
                                                "math_score_x" : "Average Math Score", 
                                            "reading_score" : "Average Reading Score"})
size_summary = pd.merge(size_summary, passing_totals, on="Size Group", how="outer")
size_summary["% Passing Math"] = round(size_summary["passing_math"]/size_summary["Total Students"] * 100, 2)
size_summary["% Passing Reading"] = round(size_summary["passing_reading"]/size_summary["Total Students"] * 100, 2)
size_summary["% Passing Overall"] = round(size_summary["passing_overall"]/size_summary["Total Students"] * 100, 2)
size_summary = size_summary.filter( ["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Passing Overall"])
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Size Group,,,,,
Small (<1000),83.828654,83.974082,93.95,96.04,90.14
Medium (1000 - 2000),83.372682,83.867989,93.62,96.77,90.62
Large (2000 - 5000),77.477597,81.198674,68.65,82.13,56.57


## Scores by School Type

* Perform the same operations as above, based on school type

In [74]:
school_data_complete.head()
workingfile = school_data_complete.filter(["type", "math_score", "reading_score", "size"])
workingfile["passing_math"] = np.where(working_file["math_score"] >= 70, 1, 0)
workingfile["passing_reading"]= np.where(working_file["reading_score"] >= 70, 1, 0)
workingfile["passing_overall"] = np.where((working_file["math_score"] >= 70) & (working_file["reading_score"] >= 70), 1, 0)


groupedfile = workingfile.groupby("type")

average_scores = groupedfile[["math_score", "reading_score"]].mean()
total_students = groupedfile["math_score"].count()
passing_totals = groupedfile[["passing_math", "passing_reading", "passing_overall"]].sum()

type_summary = pd.merge(average_scores, total_students, on="type", how="outer")
type_summary = type_summary.rename (columns={"math_score_y" : "Total Students", 
                                            "math_score_x" : "Average Math Score", 
                                            "reading_score" : "Average Reading Score"})
type_summary = pd.merge(type_summary, passing_totals, on="type", how="outer")
type_summary["% Passing Math"] = round(type_summary["passing_math"]/type_summary["Total Students"] * 100, 2)
type_summary["% Passing Reading"] = round(type_summary["passing_reading"]/type_summary["Total Students"] * 100, 2)
type_summary["% Passing Overall"] = round(type_summary["passing_overall"]/type_summary["Total Students"] * 100, 2)
type_summary = type_summary.filter( ["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Passing Overall"])
type_summary



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
type,,,,,
Charter,83.406183,83.902821,93.70,96.65,90.56
District,76.987026,80.962485,66.52,80.91,53.70
